In [4]:
import skimage
from skimage import io, transform
from skimage.color import rgb2gray
import numpy as np
from scipy.signal import convolve2d
import math
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import cross_val_score, StratifiedKFold, train_test_split
from sklearn.grid_search import GridSearchCV
import pandas as pd
from sklearn.metrics import accuracy_score
import pickle
import os
from random import choice
from skimage.transform import hough_line, hough_line_peaks
from sklearn.decomposition import PCA

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['choice']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
D_x = np.array([[-1,0,1]])
D_y = D_x.T
S_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
S_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
print D_x
print D_y
print S_x
print S_y

[[-1  0  1]]
[[-1]
 [ 0]
 [ 1]]
[[-1  0  1]
 [-2  0  2]
 [-1  0  1]]
[[ 1  2  1]
 [ 0  0  0]
 [-1 -2 -1]]


In [6]:
def to_common_size(image, common_size=60.):
    return skimage.transform.resize(image, (common_size,common_size))

In [7]:
def extract_hog(image_name, filters=[D_x, D_y], cell_rows=12, cell_colls=12, bin_count=20, 
                block_rows=8, block_coll=8, eps=0.01, folder_name='train/'):
    
    image = io.imread(folder_name + image_name)
    img = rgb2gray(image)
    #img = skimage.exposure.equalize_hist(img, nbins=bin_count)
    img = np.array(to_common_size(img))
    
    assert img.shape == (60, 60)
    
    conv_x = convolve2d(img, filters[0], mode='same')
    conv_y = convolve2d(img, filters[1], mode='same')
    
    #io.imshow_collection([image, img, conv_x, conv_y])
    #io.show()
    
    G = np.sqrt(conv_x ** 2 + conv_y ** 2)
    Q = np.arctan2(conv_x, conv_y)
    
    cell_size_row = 5 #G.shape[0] / cell_rows
    cell_size_coll = 5 #G.shape[1] / cell_colls
    hist_cells = np.zeros(shape=(cell_rows, cell_colls, bin_count))

    for i in range(cell_rows):
        for j in range(cell_colls):
            #print i * cell_size_row , (i + 1) * cell_size_row + 1,\
            #                                   j * cell_size_coll , (j + 1) * cell_size_coll + 1
            hist_cells[i, j] = np.histogram(Q[i * cell_size_row : (i + 1) * cell_size_row + 1,
                                               j * cell_size_coll : (j + 1) * cell_size_coll + 1],
                                             bins=bin_count,
                                             weights=G[i * cell_size_row : (i + 1) * cell_size_row + 1,
                                                       j * cell_size_coll : (j + 1) * cell_size_coll + 1],
                                            range=(-np.pi, np.pi))[0]

    #times_to_zip_rows = cell_rows / block_rows
    #times_to_zip_colls = cell_colls / block_coll
    
    hist_blocks = np.zeros(shape=(block_rows, block_coll, bin_count))
    for i in range(block_rows):
        for j in range(block_coll):
            
            #print i , (i + 2) , j , (j + 2) 
            hist_blocks[i,j] = np.sum(hist_cells[i : i + 2 + 1, 
                                                 j : j + 2 + 1, :], axis=(0,1))
            
            #print hist_blocks[i,j].sum() / np.sqrt(hist_blocks[i,j].sum() ** 2 + eps), hist_blocks[i,j].shape
            hist_blocks[i,j] = hist_blocks[i,j] / np.sqrt(hist_blocks[i,j].sum() ** 2 + eps)
            #print hist_blocks[i,j].sum()
            
    del img
    del image, hist_cells
    del conv_x, conv_y

    v = hist_blocks.ravel()
    return v

# Препроцессинг

In [8]:
train_data = pd.read_csv('train/gt.csv')
train_data.head()

,filename,class_id
0,00000.png,0
1,00001.png,0
2,00002.png,0
3,00003.png,0
4,00004.png,0


In [6]:
filenames = list()

for i in range(2):
    filenames.append(choice(train_data.filename))
    print filenames[i]
    v = extract_hog(filenames[i])

02775.png
19576.png


In [6]:
X = map(lambda s: extract_hog(s), train_data.filename)

NameError: name 'train_data' is not defined

In [8]:
y = np.array(train_data.class_id)
print y.shape
kf = StratifiedKFold(y, n_folds=3)

(39209,)


In [9]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y)

In [10]:
svc = SVC(C=100).fit(X_tr, y_tr)
accuracy_score(y_val, svc.predict(X_val))

0.97592573701927976

In [12]:
param_grid = {'C': [1, 10, 100, 1000], 'gamma': ['auto', 0.001, 0.0001]}
gr_params = GridSearchCV(SVC(), param_grid, cv=kf, n_jobs=3)
gr_params.fit(X,y)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 0  0 ..., 42 42], n_folds=3, shuffle=False, random_state=None),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'C': [1, 10, 100, 1000], 'gamma': ['auto', 0.001, 0.0001]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [5]:
pca = PCA(n_components=200)
X_transform = pca.fit_transform(X)

NameError: name 'X' is not defined

In [13]:
gr_params.best_params_

{'C': 1000, 'gamma': 'auto'}

In [14]:
gr_params.best_score_

0.92988854599709247

In [87]:
with open('images_processed_10_10_15.pickle', 'wb') as f:
    pickle.dump(X, f)

# Тестирование моделей

In [ ]:
with open('images_processed_20_20_30.pickle', 'rb') as f:
    X = pickle.load(f)

In [20]:
y = np.array(train_data.class_id)

In [13]:
X_train, X_test,y_train, y_test = train_test_split(X, y)

In [15]:
def fit_and_classify(X_train, y_train, X_test, y_test, **params):
    svc_clf = SVC(**params)
    print svc_clf
    svc_clf.fit(X_train, y_train)
    
    return accuracy_score(y_test, svc_clf.predict(X_test))

In [65]:
fit_and_classify(X_train, y_train, X_test, y_test, C=200)

SVC(C=200, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


0.80658981944302766

In [16]:
fit_and_classify(X_train, y_train, X_test, y_test, C=100)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


0.81638273997755784

In [22]:
svc = SVC(C=100.)
svc.fit(X, y)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
with open('model_15_15_15.pickle', 'wb') as f:
    pickle.dump(svc, f)

In [17]:
test_names = sorted(os.listdir('test'))
test_names[:3], test_names[-3:]

(['00000.png', '00001.png', '00002.png'],
 ['12339.png', '12340.png', '12341.png'])

In [18]:
X_test = map(lambda s: extract_hog(s, folder_name='test/'), test_names)

In [19]:
predictions = gr_params.best_estimator_.predict(X_test)

In [20]:
pred_pd = pd.DataFrame(columns=['filename', 'class_id'])
pred_pd.filename = test_names
pred_pd.class_id = predictions

In [21]:
pred_pd.head()

,filename,class_id
0,00000.png,6
1,00001.png,3
2,00002.png,1
3,00003.png,15
4,00004.png,26


In [22]:
pred_pd.to_csv('solution_4.csv', index=False)

In [60]:
train_data.head()

,filename,class_id
0,00000.png,0
1,00001.png,0
2,00002.png,0
3,00003.png,0
4,00004.png,0
